In [1]:
import math
import random
import librosa
import os
import soundfile as sf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as ms
from tqdm import tqdm
import pickle
import pandas as pd
import scipy
from scipy.io import wavfile
import scipy.fftpack as fft
from scipy.signal import get_window
import IPython.display as ipd

import librosa.display
ms.use('seaborn-muted')
%matplotlib inline

In [4]:
#columns = ['wav_file', 'label', 'sig_mean', 'sig_std', 'rmse_mean', 'rmse_std', 'silence', 'harmonic', 'auto_corr_max', 'auto_corr_std','mel']
columns = ['wav_file', 'label', 'mel']
df_features = pd.DataFrame(columns=columns)

emotion_dict = {'ang': 0,
                'hap': 1,
                'exc': 2,
                'sad': 3,
                'fru': 4,
                'fea': 5,
                'sur': 6,
                'neu': 7,
                'xxx': 8,
                'oth': 8}

data_dir = 'pre-processed/'
labels_path = '{}df_iemocap.csv'.format(data_dir)
audio_vectors_path = '{}audio_vectors_'.format(data_dir)

In [5]:
def normalize_audio(audio):
    audio = audio / np.max(np.abs(audio))
    return audio

def frame_audio(audio, FFT_size=512, hop_size=10, sample_rate=44100):
    # hop_size in ms
    
    audio = np.pad(audio, int(FFT_size / 2), mode='reflect')
    frame_len = np.round(sample_rate * hop_size / 1000).astype(int)
    #frame_num = int((len(audio) - FFT_size) / frame_len) + 1
    #Improve this to not missout on frames
    frame_num = 300
    frames = np.zeros((frame_num,FFT_size))
    #frames_remaining = np.zeros((300,FFT_size))
    
    for n in range(frame_num):
        frames[n] = audio[n*frame_len:n*frame_len+FFT_size]
        
#     for n in range(300,frame_num):
#         frames_remaining[n-300] = audio[n*frame_len:n*frame_len+FFT_size]
    
    return frames

def get_filters(filter_points, FFT_size):
    filters = np.zeros((len(filter_points)-2,int(FFT_size/2+1)))
    
    for n in range(len(filter_points)-2):
        filters[n, filter_points[n] : filter_points[n + 1]] = np.linspace(0, 1, filter_points[n + 1] - filter_points[n])
        filters[n, filter_points[n + 1] : filter_points[n + 2]] = np.linspace(1, 0, filter_points[n + 2] - filter_points[n + 1])
    
    return filters

def freq_to_mel(freq):
    return 2595.0 * np.log10(1.0 + freq / 700.0)

def met_to_freq(mels):
    return 700.0 * (10.0**(mels / 2595.0) - 1.0)

def get_filter_points(fmin, fmax, mel_filter_num, FFT_size, sample_rate=44100):
    fmin_mel = freq_to_mel(fmin)
    fmax_mel = freq_to_mel(fmax)
    
    #print("MEL min: {0}".format(fmin_mel))
    #print("MEL max: {0}".format(fmax_mel))
    
    mels = np.linspace(fmin_mel, fmax_mel, num=mel_filter_num+2)
    freqs = met_to_freq(mels)
    
    return np.floor((FFT_size + 1) / sample_rate * freqs).astype(int), freqs


def generate_mels(audio, sr):
    audio = normalize_audio(y)
    
    hop_size = 10 #ms
    FFT_size = 512

    audio_framed = frame_audio(audio, FFT_size=FFT_size, hop_size=hop_size, sample_rate=sr)
    window = get_window("hann", FFT_size, fftbins=True)
    
    audio_win = audio_framed * window
    
    audio_winT = np.transpose(audio_win)

    audio_fft = np.empty((int(1 + FFT_size // 2), audio_winT.shape[1]), dtype=np.complex64, order='F')

    for n in range(audio_fft.shape[1]):
        audio_fft[:, n] = fft.fft(audio_winT[:, n], axis=0)[:audio_fft.shape[0]]

    audio_fft = np.transpose(audio_fft)
    
    audio_power = np.square(np.abs(audio_fft))
    
    freq_min = 0
    freq_high = sr / 2
    mel_filter_num = 40
    
    filter_points, mel_freqs = get_filter_points(freq_min, freq_high, mel_filter_num, FFT_size, sample_rate=44100)
    filters = get_filters(filter_points, FFT_size)
    
    # taken from the librosa library
    enorm = 2.0 / (mel_freqs[2:mel_filter_num+2] - mel_freqs[:mel_filter_num])
    filters *= enorm[:, np.newaxis]
    
    audio_filtered = np.dot(filters, np.transpose(audio_power))
    audio_log = 10.0 * np.log10(audio_filtered)
    audio_log_delta = librosa.feature.delta(audio_log)
    audio_log_delta_delta = librosa.feature.delta(audio_log_delta)
    
    return np.array([audio_log, audio_log_delta, audio_log_delta_delta])

In [ ]:
# Testing the code
# data_dir = 'pre-processed/'
# labels_df_path = '{}df_iemocap.csv'.format(data_dir)
# audio_vectors_path = '{}audio_vectors_1.pkl'.format(data_dir)

# labels_df = pd.read_csv(labels_df_path)
# audio_vectors = pickle.load(open(audio_vectors_path, 'rb'))

# # Pick a Random File
# random_file_name = list(audio_vectors.keys())[random.choice(range(len(audio_vectors.keys())))]
# y = audio_vectors[random_file_name]
# 

# plt.figure(figsize=(15,2))
# librosa.display.waveplot(y, sr=sr, max_sr=1000, alpha=0.25, color='r')

# filter = labels_df['wav_file']== random_file_name
  
# # filtering data
# print(labels_df[filter].emotion)

# import IPython.display as ipd
# yt, index = librosa.effects.trim(y)
# ipd.Audio(y, rate=sr)

In [6]:
labels_df = pd.read_csv(labels_path)
sr = 44100
for sess in (range(1, 6)):
        audio_vectors = pickle.load(open('{}{}.pkl'.format(audio_vectors_path, sess), 'rb'))
        for index, row in tqdm(labels_df[labels_df['wav_file'].str.contains('Ses0{}'.format(sess))].iterrows()):
            try:
                wav_file_name = row['wav_file']
                label = emotion_dict[row['emotion']]
                y = audio_vectors[wav_file_name]

                feature_list = [wav_file_name, label]  # wav_file, label
#                 sig_mean = np.mean(abs(y))
#                 feature_list.append(sig_mean)  # sig_mean
#                 feature_list.append(np.std(y))  # sig_std

#                 rmse = librosa.feature.rms(y + 0.0001)[0]
#                 feature_list.append(np.mean(rmse))  # rmse_mean
#                 feature_list.append(np.std(rmse))  # rmse_std

#                 silence = 0
#                 for e in rmse:
#                     if e <= 0.4 * np.mean(rmse):
#                         silence += 1
#                 silence /= float(len(rmse))
#                 feature_list.append(silence)  # silence

#                 y_harmonic = librosa.effects.hpss(y)[0]
#                 feature_list.append(np.mean(y_harmonic) * 1000)  # harmonic (scaled by 1000)

#                 # based on the pitch detection algorithm mentioned here:
#                 # http://access.feld.cvut.cz/view.php?cisloclanku=2009060001
#                 cl = 0.45 * sig_mean
#                 center_clipped = []
#                 for s in y:
#                     if s >= cl:
#                         center_clipped.append(s - cl)
#                     elif s <= -cl:
#                         center_clipped.append(s + cl)
#                     elif np.abs(s) < cl:
#                         center_clipped.append(0)
#                 auto_corrs = librosa.core.autocorrelate(np.array(center_clipped))
#                 feature_list.append(1000 * np.max(auto_corrs)/len(auto_corrs))  # auto_corr_max (scaled by 1000)
#                 feature_list.append(np.std(auto_corrs))  # auto_corr_std
                
#                 # add mel of (3x40x300)
                feature_list.append(generate_mels(y, sr))

                df_features = df_features.append(pd.DataFrame(feature_list, index=columns).transpose(), ignore_index=True)
            except Exception as e: 
                print(e)
                print('Some exception occured')

df_features.to_csv('pre-processed/audio_features.csv', index=False)

15it [00:00, 146.02it/s]

could not broadcast input array from shape (319,) into shape (512,)
Some exception occured
could not broadcast input array from shape (182,) into shape (512,)
Some exception occured
could not broadcast input array from shape (402,) into shape (512,)
Some exception occured
could not broadcast input array from shape (182,) into shape (512,)
Some exception occured
could not broadcast input array from shape (72,) into shape (512,)
Some exception occured
could not broadcast input array from shape (72,) into shape (512,)
Some exception occured
could not broadcast input array from shape (72,) into shape (512,)
Some exception occured
could not broadcast input array from shape (240,) into shape (512,)
Some exception occured
could not broadcast input array from shape (173,) into shape (512,)
Some exception occured
could not broadcast input array from shape (367,) into shape (512,)
Some exception occured


44it [00:00, 118.42it/s]

could not broadcast input array from shape (72,) into shape (512,)
Some exception occured
could not broadcast input array from shape (72,) into shape (512,)
Some exception occured
could not broadcast input array from shape (72,) into shape (512,)
Some exception occured
could not broadcast input array from shape (72,) into shape (512,)
Some exception occured


90it [00:00, 133.40it/s]

...

could not broadcast input array from shape (72,) into shape (512,)
Some exception occured
could not broadcast input array from shape (72,) into shape (512,)
Some exception occured
could not broadcast input array from shape (72,) into shape (512,)
Some exception occured
could not broadcast input array from shape (72,) into shape (512,)
Some exception occured
could not broadcast input array from shape (72,) into shape (512,)
Some exception occured
could not broadcast input array from shape (72,) into shape (512,)
Some exception occured
could not broadcast input array from shape (72,) into shape (512,)
Some exception occured
could not broadcast input array from shape (72,) into shape (512,)
Some exception occured
could not broadcast input array from shape (72,) into shape (512,)
Some exception occured
could not broadcast input array from shape (72,) into shape (512,)
Some exception occured


2151it [00:20, 102.99it/s]

could not broadcast input array from shape (72,) into shape (512,)
Some exception occured
could not broadcast input array from shape (72,) into shape (512,)
Some exception occured
could not broadcast input array from shape (72,) into shape (512,)
Some exception occured
could not broadcast input array from shape (72,) into shape (512,)
Some exception occured
could not broadcast input array from shape (72,) into shape (512,)
Some exception occured
could not broadcast input array from shape (73,) into shape (512,)
Some exception occured
could not broadcast input array from shape (72,) into shape (512,)
Some exception occured


2170it [00:20, 105.07it/s]


could not broadcast input array from shape (72,) into shape (512,)
Some exception occured
could not broadcast input array from shape (72,) into shape (512,)
Some exception occured
could not broadcast input array from shape (72,) into shape (512,)
Some exception occured
could not broadcast input array from shape (72,) into shape (512,)
Some exception occured
could not broadcast input array from shape (72,) into shape (512,)
Some exception occured
could not broadcast input array from shape (72,) into shape (512,)
Some exception occured
could not broadcast input array from shape (72,) into shape (512,)
Some exception occured


In [ ]:
labels_df = pd.read_csv(labels_path)
sr = 44100
for sess in (range(1, 2)):
        audio_vectors = pickle.load(open('{}{}.pkl'.format(audio_vectors_path, sess), 'rb'))
        for index, row in tqdm(labels_df[labels_df['wav_file'].str.contains('Ses0{}'.format(sess))].iterrows()):
            try:
                wav_file_name = row['wav_file']
                label = emotion_dict[row['emotion']]
                y = audio_vectors[wav_file_name]

                feature_list = [wav_file_name, label]  # wav_file, label
                sig_mean = np.mean(abs(y))
                feature_list.append(sig_mean)  # sig_mean
                feature_list.append(np.std(y))  # sig_std

                rmse = librosa.feature.rms(y + 0.0001)[0]
                feature_list.append(np.mean(rmse))  # rmse_mean
                feature_list.append(np.std(rmse))  # rmse_std

                silence = 0
                for e in rmse:
                    if e <= 0.4 * np.mean(rmse):
                        silence += 1
                silence /= float(len(rmse))
                feature_list.append(silence)  # silence

                y_harmonic = librosa.effects.hpss(y)[0]
                feature_list.append(np.mean(y_harmonic) * 1000)  # harmonic (scaled by 1000)

                # based on the pitch detection algorithm mentioned here:
                # http://access.feld.cvut.cz/view.php?cisloclanku=2009060001
                cl = 0.45 * sig_mean
                center_clipped = []
                for s in y:
                    if s >= cl:
                        center_clipped.append(s - cl)
                    elif s <= -cl:
                        center_clipped.append(s + cl)
                    elif np.abs(s) < cl:
                        center_clipped.append(0)
                auto_corrs = librosa.core.autocorrelate(np.array(center_clipped))
                feature_list.append(1000 * np.max(auto_corrs)/len(auto_corrs))  # auto_corr_max (scaled by 1000)
                feature_list.append(np.std(auto_corrs))  # auto_corr_std
                
                # add mel of (3x40x300)
                feature_list.append(np.std(generate_mels(y, sr)))

                df_features = df_features.append(pd.DataFrame(feature_list, index=columns).transpose(), ignore_index=True)
            except Exception as e: 
                print(e)
                print('Some exception occured')

df_features.to_csv('pre-processed/audio_features.csv', index=False)